In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import requests
from PIL import Image
from io import BytesIO
import cv2
import pandas as pd
import numpy as np
import os
import zipfile
import keras 
import tensorflow as tf
from keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from skimage import io, transform

# import keras library 
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library

# import sklearn
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score

# For graph 
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

Using TensorFlow backend.


In [2]:
loc_ref = '/content/drive/My Drive/59760_840806_bundle_archive.zip'
zip_ref = zipfile.ZipFile(loc_ref,'r')
zip_ref.extractall('/animals10')
zip_ref.close()

In [3]:
translate = {"cane": "Dog", "cavallo": "Horse", "elefante": "Elephant", "farfalla": "Butterfly", "gallina": "Chicken", "gatto": "Cat", "mucca": "Cow", "pecora": "Sheep", "scoiattolo": "Squirrel", "ragno": "Spider"}

In [4]:
foldernames = os.listdir('/animals10/raw-img/')
files, files2, target, target2 = [], [], [], []

for i, folder in enumerate(foldernames):
    filenames = os.listdir("/animals10/raw-img/" + folder);
    count = 0
    for file in filenames:
        if count < 1400:
            files.append("/animals10/raw-img/" + folder + "/" + file)
            target.append(translate[folder])
        else:
            files2.append("/animals10/raw-img/" + folder + "/" + file)
            target2.append(translate[folder])
        count += 1

df = pd.DataFrame({'Filepath':files, 'Target':target})
dft = pd.DataFrame({'Filepath':files2, 'Target':target2})
train, test = train_test_split(df, test_size=0.2857, random_state = 0)

In [5]:
datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        samplewise_center = True,
        )

datagen_test = ImageDataGenerator(rescale=1./255, samplewise_center = True)

train_flow = datagen.flow_from_dataframe(train, x_col = 'Filepath', y_col = 'Target', target_size=(224, 224), interpolation = 'lanczos', validate_filenames = False)
test_flow = datagen_test.flow_from_dataframe(test, x_col = 'Filepath', y_col = 'Target', target_size=(224, 224), interpolation = 'lanczos', validate_filenames = False)

Found 10000 non-validated image filenames belonging to 10 classes.
Found 4000 non-validated image filenames belonging to 10 classes.


In [6]:


pretrained_model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
for layer in pretrained_model.layers:
  layer.trainable=False
x = pretrained_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(10,activation='softmax')(x)
model =keras.models.Model(pretrained_model.input,outputs=x)
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
model.fit_generator(train_flow,epochs=5,validation_data=test_flow)

Epoch 1/5
313/313 [==============================] - 2473s 8s/step - loss: 0.3681 - accuracy: 0.9563 - val_loss: 1.8530 - val_accuracy: 0.8192
Epoch 2/5
313/313 [==============================] - 2475s 8s/step - loss: 0.3326 - accuracy: 0.9692 - val_loss: 1.8544 - val_accuracy: 0.8212
Epoch 3/5
313/313 [==============================] - 2439s 8s/step - loss: 0.3141 - accuracy: 0.9728 - val_loss: 1.8761 - val_accuracy: 0.8213
Epoch 4/5
313/313 [==============================] - 2563s 8s/step - loss: 0.3006 - accuracy: 0.9746 - val_loss: 2.5881 - val_accuracy: 0.8211
Epoch 5/5
313/313 [==============================] - 2690s 9s/step - loss: 0.2953 - accuracy: 0.9753 - val_loss: 2.5432 - val_accuracy: 0.8181


In [8]:
score = model.evaluate(test_flow, verbose=0)
print('Test data accuracy:', score[1])

Test data accuracy: 0.8181498646736145
